In [22]:
import os

# Path to the Invoice folder
invoice_folder = 'Invoices'

# List to store PDF files
pdf_files = []

# Iterate over all files in the Invoice folder
for root, dirs, files in os.walk(invoice_folder):
    for file in files:
        if file.endswith('.PDF'):
            pdf_files.append(os.path.join(root, file))

# Print the number of PDF files
print(f"Number of PDF files: {len(pdf_files)}")

Number of PDF files: 111


In [46]:
import fitz  # PyMuPDF
import pandas as pd
from IPython.display import display

def extract_table_from_pdf(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)

    text = ""
    # Iterate over the pages and extract text
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text("text")

    numbers = []
    descriptions = []
    cat = []
    quant_ord = []
    quant_ship = []
    unit_price = []
    unit_weight = []
    total = []

    i = 0
    splittext = text.split("\n")
    for k in splittext:
        if(i<8):
            i += 1
            pass
        else:
            j = k.strip()
            if(j == 'Total Order Weight:'):
                break
            mod = i % 8
            if mod == 0:
                numbers.append(j)          # First position: Item number or identifier
            elif mod == 1:
                descriptions.append(j)     # Second position: Description
            elif (mod == 2 and ".00" not in j):
                cat.append(j)              # Third position: Category
            elif (mod == 2 and ".00" in j):
                cat.append(None)
                mod += 1
                i += 1

            if mod == 3:
                quant_ord.append(j)        # Fourth position: Quantity ordered
            elif mod == 4:
                quant_ship.append(j)       # Fifth position: Quantity shipped
            elif mod == 5:
                unit_price.append(j)       # Sixth position: Unit price
            elif mod == 6:
                unit_weight.append(j)      # Seventh position: Unit weight
            elif mod == 7:
                total.append(j)            # Eighth position: Total amount
            
            i += 1

    lines = text.splitlines()

    # Find the line with "Invoice Date" and get the next line
    invoice_date = None
    for i, line in enumerate(lines):
        if "Invoice Date:" in line:
            invoice_date = lines[i + 1].strip()
            break

    data = {
        "Number": numbers,
        "Description": descriptions,
        "Category": cat,
        "Quantity Ordered": quant_ord,
        "Quantity Shipped": quant_ship,
        "Unit Price": unit_price,
        "Unit Weight": unit_weight,
        "Total": total
    }

    # Convert dictionary to DataFrame
    df = pd.DataFrame(data)
    df['Invoice Date'] = invoice_date

    return df

combined_df = pd.DataFrame()

# Iterate through each PDF, extract the table, and add it to the combined dataframe
for pdf_path in pdf_files:  # Ensure pdf_files contains the paths to your PDF files
    try:
        # Assuming extract_table_from_pdf is a function that extracts a table from a PDF and returns a dataframe
        df = extract_table_from_pdf(pdf_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    except:
        print(f"Failed to extract from {pdf_path}")

# Save the combined dataframe to an Excel file
combined_df.to_excel("CombinedInvoice.xlsx", index=False)